In [14]:
import pandas as pd
import numpy as np
import cv2
import os

In [15]:
home_dir = os.getenv("HOME")
raw_data = pd.read_csv(home_dir + '/Desktop/DataPacket/data.csv')

In [16]:
raw_data.tail()

,laser0,laser1,laser2,laser3,laser4,laser5,laser6,laser7,laser8,laser9,...,laser172,laser173,laser174,laser175,laser176,laser177,laser178,laser179,speed,steering_angle
1544,4.816050,4.808475,4.825094,4.819104,4.814940,4.821913,4.838631,4.843539,4.870230,4.888047,...,1.497087,1.492515,1.470439,1.484231,1.482382,1.471577,1.472082,1.477870,1.5,-0.046
1545,4.840387,4.833663,4.838636,4.834896,4.844656,4.839835,4.864824,4.882144,4.882463,4.923291,...,1.458358,1.477810,1.470574,1.468011,1.470980,1.464960,1.466421,1.502800,1.5,-0.046
1546,4.849373,4.855845,4.851895,4.864667,4.876279,4.877153,4.899023,4.906648,4.911956,4.924626,...,1.483265,1.450763,1.456082,1.456520,1.464542,1.453171,1.446331,1.446458,1.5,-0.046
1547,4.849373,4.855845,4.851895,4.864667,4.876279,4.877153,4.899023,4.906648,4.911956,4.924626,...,1.483265,1.450763,1.456082,1.456520,1.464542,1.453171,1.446331,1.446458,1.5,-0.046
1548,4.849373,4.855845,4.851895,4.864667,4.876279,4.877153,4.899023,4.906648,4.911956,4.924626,...,1.483265,1.450763,1.456082,1.456520,1.464542,1.453171,1.446331,1.446458,1.5,-0.046


In [17]:
laser_data = raw_data.drop(["steering_angle", "speed"], axis=1)

laser_data = laser_data.values

maximum_laser_range = 12

for i in range(len(laser_data)):
    for j in range(len(laser_data[i])):
        if np.isinf(laser_data[i][j]):
            laser_data[i][j] = 12.0
            
laser_data = laser_data / 12

from sklearn.model_selection import train_test_split

X_laser_train, X_laser_validation = train_test_split(laser_data, test_size=0.2)

In [18]:
steering_angle = raw_data["steering_angle"].values

'''
from sklearn.preprocessing import scale
steering_angle = scale( steering_angle, axis=0, with_mean=True, with_std=True, copy=True )

steering_angle
'''

'\nfrom sklearn.preprocessing import scale\nsteering_angle = scale( steering_angle, axis=0, with_mean=True, with_std=True, copy=True )\n\nsteering_angle\n'

In [19]:
steering_angle = steering_angle.transpose()

steering_angle

array([ 0.   ,  0.   ,  0.   , ..., -0.046, -0.046, -0.046])

In [20]:
img_input = []
h = 240
w = 320
c = 3
img_num = len(raw_data) + 1  # Because csv lenght start with 0

for i in range(1, img_num):
    img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", c)
    img = cv2.resize(img, (w, h))
    
    img_input.append(img)

In [21]:
img_input = np.asarray(img_input).astype(np.float32) / 255.0

In [22]:
img_input.shape

(1549, 240, 320, 3)

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(img_input, steering_angle, test_size=0.2, random_state=22)

del img_input
del steering_angle

In [24]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Concatenate, Input
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers.core import Activation

Using TensorFlow backend.


In [25]:
X_train.shape

(1239, 240, 320, 3)

In [26]:
y_train.shape

(1239,)

In [27]:
laser_data.shape[1]

180

In [32]:
model = Sequential()

model.add(Conv2D(filters = 2, kernel_size = (3,3),padding = 'same',activation ='relu'))
model.add(Conv2D(filters = 4, kernel_size = (3,3),padding = 'same',activation ='relu'))
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 24, kernel_size = (7,7),padding = 'same',activation ='relu'))
model.add(Flatten())

image_input = Input(shape=(h, w, c))
flatten_image_outputs = model(image_input)

laser_input = Input(shape = (laser_data.shape[1],))
concatenatedFeatures = Concatenate(axis = 1)([laser_input, flatten_image_outputs])

dense = Dense(128, activation = "relu")(concatenatedFeatures)
dense = Dense(64, activation = "relu")(dense)
dense = Dense(16, activation = "relu")(dense)
main_output = Dense(1, activation = "softmax")(dense)

drive_model = Model(inputs=[image_input,laser_input], outputs=main_output)

drive_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [33]:
hs = drive_model.fit([X_train, X_laser_train], y_train, validation_data=([X_validation, X_laser_validation], y_validation), batch_size=64, epochs=5)

Train on 1239 samples, validate on 310 samples
Epoch 1/5
1239/1239 [==============================] - 220s 177ms/step - loss: 0.9991 - acc: 0.0000e+00 - val_loss: 0.9899 - val_acc: 0.0000e+00
Epoch 2/5
 192/1239 [===>..........................] - ETA: 2:46 - loss: 0.9976 - acc: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
import rospkg
import rospy

rospack = rospkg.RosPack()
package_path = rospack.get_path('angelshark_deeplearning')

In [ ]:
import json
import matplotlib.pylab as plt
import random
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# Save model weights and json.
model.save_weights(package_path + '/model/model.h5')
model_json = model.to_json()
with open(package_path + '/model/model.json', 'w') as outfile:
    json.dump(model_json, outfile)

# Train and validation loss chart
print(hs.history.keys())

plt.plot(hs.history['loss'])
plt.plot(hs.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()
